# LDA Bahasa Indonesia

Instalasi beberapa library yang diperlukan, untuk NLTK dan GENSIM sudah include di Colab

In [ ]:
pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.1 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
pip install pyldavis==3.2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 7.1 MB/s 
  Created wheel for pyldavis: filename=pyLDAvis-3.2.1-py2.py3-none-any.whl size=136187 sha256=e8f236b048f670fb17e783487fce84c998eb9b6de8642eec8043d7b3e2b25c44
  Stored in directory: /root/.cache/pip/wheels/c6/ee/a6/7c17a63623f940dff0b9cbd7e48a27543f088fa55a7d2b62d0
Successfully built pyldavis


In [ ]:
pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 7.7 MB/s 


In [ ]:
pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 830 kB 6.3 MB/s 
     |████████████████████████████████| 291 kB 64.2 MB/s 
     |████████████████████████████████| 1.6 MB 49.1 MB/s 
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16322 sha256=97ca547d2685b031abaf0055ecd6ace8f890c6ecfe0985b8742c6c41901d5e47
  Stored in directory: /root/.cache/pip/wheels/29/a7/0e/3a8f17ac69d759e1e93647114bc9bdc95957e5b0cbfd405205
Successfully built swifter
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


# Preparing Data

In [ ]:
import pandas as pd
import numpy as np
import nltk

In [ ]:
dataSB = pd.read_csv('KAPOLDA1.csv') #lokasi file

dataSB.head()

,clean_text
0,8 irjen pol johanis asadoma dilantik menjadi k...
1,kapolri jenderal listyo sigit prabowo meminta ...
2,kapolri lantik irjen suharyono jadi kapolda su...
3,tuntut kembalikan kepercayaan publik kapolri t...
4,yuk baca informasi lengkapnya di


In [ ]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas

dataSB['data_bersih'] = dataSB['clean_text'].str.lower()

print('Case Folding Result : \n')
print(dataSB['data_bersih'].head(5))

Case Folding Result : 

0    8 irjen pol johanis asadoma dilantik menjadi k...
1    kapolri jenderal listyo sigit prabowo meminta ...
2    kapolri lantik irjen suharyono jadi kapolda su...
3    tuntut kembalikan kepercayaan publik kapolri t...
4                     yuk baca informasi lengkapnya di
Name: data_bersih, dtype: object


# Tokenizing

Menghapus karakter-karakter yang tidak penting dari text

In [ ]:
import string
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

In [ ]:
# ------ Tokenizing ---------

nltk.download('punkt')

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

dataSB['data_bersih'] = dataSB['data_bersih'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

dataSB['data_bersih'] = dataSB['data_bersih'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

dataSB['data_bersih'] = dataSB['data_bersih'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

dataSB['data_bersih'] = dataSB['data_bersih'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

dataSB['data_bersih'] = dataSB['data_bersih'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

dataSB['data_bersih'] = dataSB['data_bersih'].apply(remove_singl_char)

# NLTK word tokenize
def word_tokenize_wrapper(text):
    return word_tokenize(text)

dataSB['data_tokens'] = dataSB['data_bersih'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n')
print(dataSB['data_tokens'].head())

Tokenizing Result : 

0    [irjen, pol, johanis, asadoma, dilantik, menja...
1    [kapolri, jenderal, listyo, sigit, prabowo, me...
2    [kapolri, lantik, irjen, suharyono, jadi, kapo...
3    [tuntut, kembalikan, kepercayaan, publik, kapo...
4               [yuk, baca, informasi, lengkapnya, di]
Name: data_tokens, dtype: object


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

dataSB['data_tokens_fdist'] = dataSB['data_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n')
print(dataSB['data_tokens_fdist'].head().apply(lambda x : x.most_common()))

Frequency Tokens : 

0    [(irjen, 2), (pol, 2), (dilantik, 2), (menjadi...
1    [(kapolri, 1), (jenderal, 1), (listyo, 1), (si...
2    [(kapolri, 1), (lantik, 1), (irjen, 1), (suhar...
3    [(tuntut, 1), (kembalikan, 1), (kepercayaan, 1...
4    [(yuk, 1), (baca, 1), (informasi, 1), (lengkap...
Name: data_tokens_fdist, dtype: object


# Stopwords
Membersihkan teks dari kata-kata yang termasuk ke dalam stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo',
                       'kalo', 'amp', 'biar', 'bikin', 'bilang',
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih',
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya',
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't',
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah', 'bisnis', 'pandemi', 'indonesia'])

# convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

dataSB['data_tokens_WSW'] = dataSB['data_tokens'].apply(stopwords_removal)

print(dataSB['data_tokens_WSW'].head())

0    [irjen, pol, johanis, asadoma, dilantik, kapol...
1    [kapolri, jenderal, listyo, sigit, prabowo, ke...
2    [kapolri, lantik, irjen, suharyono, kapolda, s...
3    [tuntut, kembalikan, kepercayaan, publik, kapo...
4                   [yuk, baca, informasi, lengkapnya]
Name: data_tokens_WSW, dtype: object


# Normalisasi
Mengganti kata-kata tertentu dengan kata lain yang lebih tepat

In [ ]:
normalizad_word = pd.read_excel('slang.xlsx') #lokasi file

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1]

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

dataSB['textdata_normalized'] = dataSB['data_tokens_WSW'].apply(normalized_term)

dataSB['textdata_normalized'].head(10)

0    [irjen, pol, johanis, asadoma, dilantik, kapol...
1    [kapolri, jenderal, listyo, sigit, prabowo, ke...
2    [kapolri, lantik, irjen, suharyono, kapolda, s...
3    [tuntut, kembalikan, kepercayaan, publik, kapo...
4                   [ayo, baca, informasi, lengkapnya]
5                   [ayo, baca, informasi, lengkapnya]
6    [tertangkap, peredaran, narkotika,psikotropika...
7                  [kapolri, lantik, perwira, kapolda]
8    [tertangkap, peredaran, narkotika,psikotropika...
9                 [kapolri, lantik, sembilan, kapolda]
Name: textdata_normalized, dtype: object

In [ ]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in dataSB['textdata_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

print(len(term_dict))


1424


In [ ]:
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)

    # untuk melihat hasilnya silahkan jalankan baris di bawah ini
    # print(term,":" ,term_dict[term])

In [ ]:
# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

dataSB['textdata_tokens_stemmed'] = dataSB['textdata_normalized'].swifter.apply(get_stemmed_term)

print(dataSB['textdata_tokens_stemmed'])

Pandas Apply:   0%|          | 0/348 [00:00<?, ?it/s]

0      [irjen, pol, johan, asadoma, lantik, kapolda, ...
1      [kapolri, jenderal, listyo, sigit, prabowo, ke...
2      [kapolri, lantik, irjen, suharyono, kapolda, s...
3      [tuntut, kembali, percaya, publik, kapolri, ev...
4                        [ayo, baca, informasi, lengkap]
                             ...                        
343    [buntut, tewas, brigadir, ujung, jenderal, cop...
344    [hewan, hutan, hukum, rimba, mangsa, kbthn, br...
345    [kapolda, aceh, stabilitas, politik, aman, kun...
346    [kapolda, gorontalo, irjen, pol, helmy, dukung...
347    [kapolda, aku, titik, terang, bunuh, anak, sub...
Name: textdata_tokens_stemmed, Length: 348, dtype: object


In [ ]:
#stopwords #2

from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["ada", "tan", "ton", "pt", "komentar", "juta", "unit", "menang", "artikel",
                       "smartphone", "tagar", "sedia", "kaskus", "seksi"])

# convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

dataSB['textdata_tokens_stemmed2'] = dataSB['textdata_tokens_stemmed'].apply(stopwords_removal)

print(dataSB['textdata_tokens_stemmed2'].head())

0    [irjen, pol, johan, asadoma, lantik, kapolda, ...
1    [kapolri, jenderal, listyo, sigit, prabowo, ke...
2    [kapolri, lantik, irjen, suharyono, kapolda, s...
3         [tuntut, percaya, publik, kapolri, evaluasi]
4                      [ayo, baca, informasi, lengkap]
Name: textdata_tokens_stemmed2, dtype: object


In [ ]:
for i in range(len(dataSB)):
        a=dataSB.iloc[i][6]
        document.append(a)

document[0:5]

['kapolda', 'mengaku', 'titik', 'terang', 'pembunuh']

In [ ]:
doc_clean = dataSB['textdata_tokens_stemmed2']
doc_clean

0      [irjen, pol, johan, asadoma, lantik, kapolda, ...
1      [kapolri, jenderal, listyo, sigit, prabowo, ke...
2      [kapolri, lantik, irjen, suharyono, kapolda, s...
3           [tuntut, percaya, publik, kapolri, evaluasi]
4                        [ayo, baca, informasi, lengkap]
                             ...                        
343    [buntut, tewas, brigadir, ujung, jenderal, cop...
344    [hewan, hutan, hukum, rimba, mangsa, kbthn, br...
345    [kapolda, aceh, stabilitas, politik, aman, kun...
346    [kapolda, gorontalo, irjen, pol, helmy, dukung...
347    [kapolda, titik, terang, bunuh, anak, subang, ...
Name: textdata_tokens_stemmed2, Length: 348, dtype: object

In [ ]:
doc_clean.to_csv("KAPOLDA[2].csv",index = False)

# LDA model using gensim
Proses topic modeling dengan LDA gensim

In [ ]:
import gensim
from gensim import corpora

dictionary = corpora.Dictionary(doc_clean)
print(dictionary)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

Dictionary<1172 unique tokens: ['asadoma', 'barat', 'irjen', 'johan', 'kapolda']...>


In [ ]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

total_topics = 3 # jumlah topik yang akan di extract
number_words = 10 # jumlah kata per topik

In [ ]:
# Running and Trainign LDA model on the document term matrix.
lda_model = Lda(doc_term_matrix, num_topics=total_topics, id2word = dictionary, passes=50)

lda_model.show_topics(num_topics=total_topics, num_words=number_words)

[(0,
  '0.100*"polisi" + 0.011*"lapor" + 0.008*"orang" + 0.005*"salah" + 0.004*"hukum" + 0.004*"oknum" + 0.004*"anak" + 0.004*"banget" + 0.003*"kantor" + 0.003*"pakai"'),
 (1,
  '0.040*"polisi" + 0.017*"na" + 0.009*"whatsapp" + 0.007*"kwa" + 0.006*"bir" + 0.005*"ve" + 0.005*"jeshi" + 0.004*"bu" + 0.003*"za" + 0.003*"adik"'),
 (2,
  '0.052*"polisi" + 0.007*"tangkap" + 0.007*"peduli" + 0.007*"aman" + 0.006*"warga" + 0.006*"bencana" + 0.006*"sigap" + 0.005*"banjir" + 0.005*"bantu" + 0.005*"laku"')]

In [ ]:
# Word Count of Topic Keywords

from collections import Counter
topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in doc_clean for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df_imp_wcount = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])
print(df_imp_wcount)

In [ ]:
df_imp_wcount.to_csv('wcount.csv', index=False)

In [ ]:
#jika simpan ke local drive
#filedisimpan='df_imp_wcount.xlsx'
#df_imp_wcount.to_excel(filedisimpan, index = False, header=True)

In [ ]:
#Dominant topic and its percentage contribution in each topic
def format_topics_sentences(ldamodel=None, corpus=doc_term_matrix, texts=document):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [ ]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=doc_term_matrix, texts=doc_clean)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
print(df_dominant_topic.head(10))

   Document_No  Dominant_Topic  Topic_Perc_Contrib  \
0            0             0.0              0.7440   
1            1             1.0              0.9546   
2            2             1.0              0.9132   
3            3             1.0              0.9575   
4            4             1.0              0.9575   
5            5             2.0              0.9141   
6            6             2.0              0.9141   
7            7             0.0              0.9374   
8            8             2.0              0.8645   
9            9             2.0              0.8645   

                                            Keywords  \
0  polisi, oknum, libat, pecat, duga, anggota, me...   
1  polisi, oknum, viral, tangkap, aniaya, polda, ...   
2  polisi, oknum, viral, tangkap, aniaya, polda, ...   
3  polisi, oknum, viral, tangkap, aniaya, polda, ...   
4  polisi, oknum, viral, tangkap, aniaya, polda, ...   
5  polisi, oknum, polri, propam, irjen, anggota, ...   
6  polisi, ok

In [ ]:
df_dominant_topic.to_csv('df_dominant_topic.csv')

In [ ]:
import pyLDAvis.gensim
import pickle
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [ ]:
import os
LDAvis_data_filepath = os.path.join('ldavis_prepared_'+str(total_topics))

In [ ]:
corpus = [dictionary.doc2bow(text) for text in doc_clean]

In [ ]:
# proses ini mungkin agak lama
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

In [ ]:
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

In [ ]:
# proses ini mungkin agak lama
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.108823  0.051870       1        1  48.076171
1     -0.002763 -0.107903       2        1  29.620900
2     -0.106060  0.056033       3        1  22.302929, topic_info=                                       Term       Freq      Total Category  \
132                                   libat  37.000000  37.000000  Default   
98                                    pecat  35.000000  35.000000  Default   
67                                    polri  27.000000  27.000000  Default   
23                                    viral  28.000000  28.000000  Default   
42                                    irjen  16.000000  16.000000  Default   
..                                      ...        ...        ...      ...   
23                                    viral   6.101154  28.064126   Topic3   
99                                    sumut   4.647488  10.788077   Topic3   
78                                     duga   6.263270  49.765124   Topic3   
37   narkotika psikotropika dan obat larang   4.737957  25.063040   Topic3   
35                                     kait   4.537160  16.781204   Topic3   

     logprob  loglift  
132  30.0000  30.0000  
98   29.0000  29.0000  
67   28.0000  28.0000  
23   27.0000  27.0000  
42   26.0000  26.0000  
..       ...      ...  
23   -5.1389  -0.0256  
99   -5.4111   0.6583  
78   -5.1127  -0.5722  
37   -5.3918  -0.1653  
35   -5.4351   0.1925  

[190 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
153       1  0.348003      aksi
153       2  0.580005      aksi
58        3  0.824692     aktif
131       1  0.923505     ampas
94        1  0.983467      ampo
...     ...       ...       ...
107       1  0.583117     warga
107       2  0.356350     warga
107       3  0.064791     warga
617       3  0.788043  wartawan
1043      2  0.931366       wil

[219 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])